In [2]:
import optuna
import os

# Ruta a tu base de datos Optuna (ajustá si es necesario)
db_path = "../../../buckets/b1/Compe_02/optuna_db/TEST 1 - Modelo Final  - Pruebo Learning Rate con funcionamiento variable.db"

# Verificar que el archivo exista
if not os.path.exists(db_path):
    raise FileNotFoundError(f"No se encontró el archivo: {db_path}")

# Crear el storage URL (formato SQLite)
storage_url = f"sqlite:///{db_path}"

# Cargar el estudio
study_name = "TEST 1 - Modelo Final  - Pruebo Learning Rate con funcionamiento variable"  # debe coincidir exactamente
study = optuna.load_study(study_name=study_name, storage=storage_url)

# Obtener el mejor trial
best_trial = study.best_trial

# Mostrar resultados
print("Número del mejor trial:", best_trial.number)
print("Valor objetivo:", best_trial.value)
print("Parámetros:")
for k, v in best_trial.params.items():
    print(f"  {k}: {v}")

# Si querés guardarlo como diccionario
mejores_params = dict(best_trial.params)
print("\nDiccionario con los mejores hiperparámetros:")
print(mejores_params)


Número del mejor trial: 4
Valor objetivo: 2147474611.2
Parámetros:
  num_leaves: 23
  min_data_in_leaf: 223
  feature_fraction: 0.6616669584635271
  bagging_fraction: 0.23994377622330532
  num_boost_round: 439
  lr_init: 0.14053552566659705
  lr_decay: 0.9124750514032693

Diccionario con los mejores hiperparámetros:
{'num_leaves': 23, 'min_data_in_leaf': 223, 'feature_fraction': 0.6616669584635271, 'bagging_fraction': 0.23994377622330532, 'num_boost_round': 439, 'lr_init': 0.14053552566659705, 'lr_decay': 0.9124750514032693}


In [ ]:
import pandas as pd

# Ruta al archivo
path = "../../../buckets/b1/datasets/competencia_02_crudo.csv.gz"

# Cargar dataset (pandas detecta el gzip automáticamente)
df = pd.read_csv(path)

# --- Información general ---
print("🔹 Dimensiones del dataset:", df.shape)
print("\n🔹 Tipos de datos:")
print(df.dtypes.value_counts())
print("\n🔹 Primeras filas:")
print(df.head())

# --- Descriptivas numéricas ---
print("\n📊 Descriptivas numéricas:")
print(df.describe().T)

# --- Descriptivas categóricas ---
print("\n🔠 Descriptivas categóricas:")
cat_cols = df.select_dtypes(include=['object', 'category']).columns
if len(cat_cols) > 0:
    print(df[cat_cols].describe().T)
else:
    print("No hay columnas categóricas.")

# --- Valores nulos ---
print("\n🚫 Porcentaje de valores nulos por columna:")
print((df.isnull().mean() * 100).sort_values(ascending=False))

🔹 Dimensiones del dataset: (4717958, 154)

🔹 Tipos de datos:
float64    91
int64      63
Name: count, dtype: int64

🔹 Primeras filas:
   numero_de_cliente  foto_mes  active_quarter  cliente_vip  internet  \
0          249221323    201901               1            0         1   
1          249227600    201901               1            0         1   
2          249227813    201901               1            0         0   
3          249231391    201901               1            0         1   
4          249244739    201901               1            0         1   

   cliente_edad  cliente_antiguedad  mrentabilidad  mrentabilidad_annual  \
0            44                  69          86.33              14610.01   
1            40                 254        2096.06              15905.41   
2            77                 156        1071.07              10295.09   
3            55                 252         979.98               7670.48   
4            64                 296        2724

In [ ]:

df["foto_mes"].value_counts()


In [ ]:
import pandas as pd
import numpy as np

# Crear df_target a partir de df
df_target = df[['numero_de_cliente','foto_mes']].copy()
df_target = df_target.sort_values(by=['numero_de_cliente','foto_mes'])

# Crear columnas que indican si el cliente sigue en los meses siguientes
df_target['enmes1'] = df_target.groupby('numero_de_cliente')['numero_de_cliente'].shift(-1)
df_target['enmes2'] = df_target.groupby('numero_de_cliente')['numero_de_cliente'].shift(-2)

# Calcular cuántos meses hay por delante (suponiendo que el último mes es 202108)
df_target['mesespordelante'] = 202108 - df_target['foto_mes']

# Clasificación ternaria
df_target['clase_ternaria'] = np.where(
    df_target['mesespordelante'] >= 2,
    np.where(
        (df_target['numero_de_cliente'] == df_target['enmes1']) &
        (df_target['numero_de_cliente'] == df_target['enmes2']),
        'CONTINUA',
        np.where(
            (df_target['numero_de_cliente'] == df_target['enmes1']) & (df_target['enmes2'].isna()),
            'BAJA+2',
            'BAJA+1'
        )
    ),
    np.where(
        (df_target['mesespordelante'] == 1) & (df_target['enmes1'].isna()),
        'BAJA+1',
        ''
    )
)

# Agregar la columna 'clase_ternaria' a df original
df = df.merge(
    df_target[['numero_de_cliente', 'foto_mes', 'clase_ternaria']],
    on=['numero_de_cliente', 'foto_mes'],
    how='left'
)

# Confirmar
print(df['clase_ternaria'].value_counts(dropna=False))
